In [ ]:
# Imports 
from __future__ import annotations

from dataclasses import dataclass
from typing import (
    Callable,
    Dict,
    Iterable,
    Mapping,
    MutableMapping,
    Optional,
    Sequence,
    Tuple,
    Union,
)

import torch
from torch import nn

from pina import Condition, LabelTensor
from pina.equation import Equation, FixedValue
from pina.model import FeedForward
from pina.operator import div, grad
from pina.solver.physics_informed_solver import PINN
from pina.solver.physics_informed_solver.rba_pinn import RBAPINN
from pina.solver.physics_informed_solver.self_adaptive_pinn import SelfAdaptivePINN
from pina.trainer import Trainer

from . import bc as bc_utils
from .geometry import PlanePatch, TowerAGeometry

In [ ]:
# Helpers to standardize 
Axes = Tuple[str, ...]
TensorLike = Union[float, int, Sequence[float], torch.Tensor, LabelTensor]
ConductivityLike = Union[TensorLike, Callable[[LabelTensor], TensorLike]]
BoundaryValueLike = Union[TensorLike, Callable[[LabelTensor], TensorLike]]

SolverType = Union[PINN, SelfAdaptivePINN, RBAPINN]
SolverClass = Union[type[PINN], type[SelfAdaptivePINN], type[RBAPINN]]

DEFAULT_AXES: Axes = ("x", "y", "z")


def _as_tensor(value: TensorLike, input_: LabelTensor, components: int = 1) -> torch.Tensor:
    """Convert ``value`` into a tensor aligned with ``input_``'s batch."""
    device = input_.device
    dtype = input_.dtype
    batch = input_.shape[0]

    if isinstance(value, LabelTensor):
        tensor = value.tensor.to(device=device, dtype=dtype)
    elif isinstance(value, torch.Tensor):
        tensor = value.to(device=device, dtype=dtype)
    elif isinstance(value, (Sequence,)):
        tensor = torch.as_tensor(value, dtype=dtype, device=device)
    else:
        tensor = torch.tensor(value, dtype=dtype, device=device)

    if tensor.ndim == 0:
        tensor = tensor.reshape(1, 1).repeat(batch, components)
    elif tensor.ndim == 1:
        if tensor.shape[0] == batch and components == 1:
            tensor = tensor.reshape(batch, 1)
        elif tensor.shape[0] == components:
            tensor = tensor.reshape(1, components).repeat(batch, 1)
        elif tensor.shape[0] == batch and components != 1:
            tensor = tensor.reshape(batch, 1).repeat(1, components)
        else:
            raise ValueError(
                f"Cannot reshape tensor of shape {tuple(tensor.shape)} to ({batch}, {components})"
            )
    elif tensor.ndim == 2:
        if tensor.shape[0] != batch:
            raise ValueError(
                f"Expected first dimension equal to batch size {batch}, got {tensor.shape}"
            )
        if tensor.shape[1] != components:
            if tensor.shape[1] == 1 and components > 1:
                tensor = tensor.repeat(1, components)
            else:
                raise ValueError(
                    f"Expected second dimension {components}, got {tensor.shape}"
                )
    else:
        raise ValueError("Expected scalar, vector, or matrix conductivity representation.")

    return tensor


def evaluate_sigma_diag(conductivity: ConductivityLike, input_: LabelTensor, axes: Axes) -> torch.Tensor:
    """Evaluate (potentially spatially varying) conductivity on ``input_`` points.

    Returns an ``(batch, len(axes))`` tensor corresponding to the diagonal of
    :math:`\\boldsymbol{\\sigma}` in the coordinate system spanned by ``axes``.
    """
    if callable(conductivity):
        value = conductivity(input_)
    else:
        value = conductivity
    sigma = _as_tensor(value, input_, components=len(axes))
    if sigma.shape[1] == 1 and len(axes) > 1:
        sigma = sigma.repeat(1, len(axes))
    return sigma


def evaluate_scalar(value: BoundaryValueLike, input_: LabelTensor) -> torch.Tensor:
    """Evaluate a scalar boundary target on ``input_`` points."""
    if callable(value):
        evaluated = value(input_)
    else:
        evaluated = value
    return _as_tensor(evaluated, input_, components=1).squeeze(-1)

## The Laplace residual 

enforces conservation of current.


At every point in space, the net current flowing out equals the current flowing in:

∇⋅𝐽=0

Using Ohm’s law for a spatially varying conductivity field 𝜎(𝑥):

 𝐽=𝜎(𝑥)∇𝜙(𝑥) 

Substituting gives the governing PDE for the potential field:

∇⋅(𝜎(𝑥) ∇𝜙(𝑥))=0

Here:

- 𝜙(𝑥) : electric potential (the quantity the neural network predicts)

- ∇𝜙 : electric field direction and magnitude

- 𝜎(𝑥) : local conductivity (possibly anisotropic or spatially varying)

- ∇⋅(𝜎∇𝜙) : measures any imbalance in current flow

The PINN learns a function 𝜙𝜃(𝑥) such that this residual approaches zero at all interior points, ensuring the predicted field satisfies the physical law of charge conservation.

In [ ]:
def build_laplace_equation(conductivity: ConductivityLike, axes: Axes = DEFAULT_AXES) -> Equation:
    """Return the ∇·(σ∇φ)  residual as a PINA equation."""

    # inner fucntion PINA will call: 
    def residual(input_: LabelTensor, output_: LabelTensor) -> LabelTensor:

        #Compute ∇𝜙∇ϕ = [∂φ/∂x, ∂φ/∂y, (∂φ/∂z)] at all sampled points(via PINA operator grad)
        grad_phi = grad(output_, input_, components=["phi"], d=list(axes))

        # Sample the conductivity field at those coordinates.
        #Returns: A tensor of shape (N, D) holding diagonal entries [σ_x, σ_y, σ_z] per point (isotropic is just repeated values).
        sigma_diag = evaluate_sigma_diag(conductivity, input_, axes) #assumes σ is diagonal

        #Calculate the current density vector 𝐽=𝜎∇𝜙
        flux_tensor = sigma_diag * grad_phi.tensor

        #wrapp the flux with labels for PINA
        flux = LabelTensor(
            flux_tensor, labels=[f"J_{ax}" for ax in axes]
        )

        #Computes ∇⋅𝐽 Returns a (N, 1) (or (N,)) residual—one scalar per point.
        divergence = div(flux, input_, components=flux.labels, d=list(axes))

        #The PDE is satisfied when this is zero at interior points. 
        #This tensor becomes the contribution to the physics loss.
        return divergence

    return Equation(residual)


# Neumann Boundary Condition — Flux Constraint

The Neumann condition specifies the normal current density crossing a boundary surface.
It enforces that the component of the current leaving (or entering) the surface equals a prescribed flux:

− −n⋅(σ(x)∇ϕ(x))=q_target(x)

where:

- 𝑛 : outward-facing unit normal vector of the surface

- 𝜎(x) : local conductivity

- ∇ϕ : potential gradient (electric field)

- q_target : desired surface flux (e.g., injected current density)
 
The negative sign ensures positive flux points outward.

During training, the PINN minimizes the residual

R(x)=−n⋅(σ∇ϕ)−q_target 

	​
so that the network’s predicted field satisfies the prescribed current flow at that boundary.



In [ ]:
def build_neumann_equation(
    patch: PlanePatch,
    target_flux_density: BoundaryValueLike,
    conductivity: ConductivityLike,
    axes: Axes = DEFAULT_AXES,
) -> Equation:
    """Return the Neumann residual enforcing ``-n·σ∇φ = target`` on a patch."""
    axis = patch.axis # Identify which coordinate axis this boundary patch is aligned with
    if axis not in axes:
        raise ValueError(f"Patch axis {axis!r} not present in axes {axes}.")
    axis_index = axes.index(axis)

     # Inner function defining the residual at each boundary point
    def residual(input_: LabelTensor, output_: LabelTensor) -> LabelTensor:

        # Compute ∂φ/∂n — the potential gradient along the patch’s normal axis
        # Here `axis` indicates which direction the normal points (e.g., 'x', 'y', or 'z')
        grad_component = grad(output_, input_, components=["phi"], d=[axis])

        # Evaluate local conductivity σ(x) along all axes and extract the relevant one
        sigma_diag = evaluate_sigma_diag(conductivity, input_, axes)
        sigma_axis = sigma_diag[:, axis_index]

        # Compute the normal component of current density:
        # -n·σ∇φ = -normal_sign * σ_axis * ∂φ/∂n
        # normal_sign adjusts for whether the patch normal points inward/outward
        normal_flux = -patch.normal_sign * sigma_axis * grad_component.tensor.squeeze(-1)

        # Evaluate the flux target ( injected current density)
        target = evaluate_scalar(target_flux_density, input_)

         # Residual → 0 when boundary current equals target flux
        return normal_flux - target

    return Equation(residual)



## Dirichlet Boundary Condition — Fixed Potential

The Dirichlet condition specifies the value of the potential itself along a boundary surface.
It enforces that the neural network’s predicted potential 𝜙(𝑥) equals a prescribed target:

𝜙(𝑥)=𝜙_target(𝑥)

where:

- 𝜙target : fixed boundary potential (can be constant or spatially varying)

The residual minimized by the PINN is simply:

R(x)=ϕ(x)−ϕ_target(x)

which drives the model to match the known potential on grounded or reference surfaces.



In [ ]:

def build_dirichlet_equation(target_value: BoundaryValueLike) -> Equation:
    """Return a Dirichlet residual enforcing ``phi = target``."""

    def residual(input_: LabelTensor, output_: LabelTensor) -> LabelTensor:
        target = evaluate_scalar(target_value, input_)
        phi = output_.extract(["phi"]).tensor.squeeze(-1)
        return phi - target

    return Equation(residual)


###  Conceptual Recap

| Builder                    | Enforces                | Equation        | Region                 |
| :------------------------- | :---------------------- | :-------------- | :--------------------- |
| `build_laplace_equation`   | Conservation of current | ∇·(σ∇φ)=0       | Interior               |
| `build_neumann_equation`   | Specified current flux  | −n·σ∇φ = target | Boundary (flux patch)  |
| `build_dirichlet_equation` | Fixed potential         | φ = target      | Boundary (fixed patch) |

In [ ]:
# Container dataclass(SurfaceConditonSummary) 
# holding metadata for each boundary condition (surface).
# Used to record which patch, condition type (Dirichlet/Neumann), and target
# value/function are applied to a specific domain boundary.
@dataclass
class SurfaceConditionSummary:
    """Metadata describing the condition attached to a boundary surface."""

    surface: PlanePatch
    condition_type: str
    target: BoundaryValueLike
    domain_name: str
    description: str


# Tower A Laplace Problem: Full PDE Assembly

In this section, we only define the class structure and variables that describe the PDE problem 

no solving or training occurs here.

This code sets up the Laplace equation and boundary constraints that together form the physics-informed loss.

The network does not train on labeled data; instead, it learns the potential field ϕ(x) by enforcing the physics that link known electrode currents (as boundary conditions) to the resulting potential distribution across the tissue, minimizing the residuals of the governing equations throughout the domain.

⎧∇⋅(σ∇ϕ)=0,       in Ω

⎨−n⋅(σ∇ϕ)=j,       on electrode k

⎩ ϕ=0,            on reference/ground surfaces

Each electrode 𝑘 is treated as an independent unit-current basis problem:
Each electrode 𝑘 defines its own basis field 𝜙_k

 by solving this system with a unit current boundary, while all other contacts are set to zero flux.
Because the Laplace operator is linear, the total potential for any stimulation pattern


ϕ(x;I)=∑​I_kϕ_k(x)

remains a valid solution.

Formally, the total loss minimized during training is:

𝐿total=𝜆PDE 𝐿PDE+𝜆BC 𝐿BC

where
PDE residual : R_PDE(x_i)=∇⋅(σ∇ϕ(x_i))

adding that residual squared to total loss: total loss: 𝐿PDE=1/𝑁Ω ∑𝑅_PDE(𝑥_𝑖)^2

The optimizer then updates the network weights so that R_PDE -> 0 everywhere



In [ ]:
class TowerALaplaceProblem(PINN.problem.__class__ if False else object):
    """PINA ``SpatialProblem`` enforcing the Tower A Laplace formulation."""

    output_variables = ["phi"] # The only field the PINN predicts φ

    def __init__(
        self,

        # Geometry object defining the interior volume and surface patches (contacts, walls, etc.)
        geometry: TowerAGeometry,

        # Conductivity field σ(x)
        conductivity: ConductivityLike,

        # Mapping of each electrode name -> applied current amplitude
        contact_currents: Mapping[str, float],

        # Optional dictionary defining extra outer boundary conditions
        outer_bc: Optional[Mapping[str, Mapping[str, BoundaryValueLike]]] = None,
        axes: Axes = DEFAULT_AXES,
    ) -> None:
        
        # Local import of SpatialProblem to avoid circular dependency:
        # This main script references PINA’s SpatialProblem, which itself may import this module indirectly.
        from pina.problem import SpatialProblem  # local import to avoid circularity

         # subclass of SpatialProblem dynamically (used only to inherit PINN problem functionality cleanly)
        class _Problem(SpatialProblem):  # type: ignore[valid-type]
            pass

        # -------------------------------
        # 1. Store geometry and materials
        # -------------------------------
        # The TowerAGeometry object contains the interior volume mesh and all electrode/outer surface patches
        self._geometry = geometry
        self._axes = axes # define which spatial coordinates are used
        self.conductivity = conductivity

        # -------------------------------
        # 2. Define contact currents FIRST
        # -------------------------------
        # Build a mapping {contact_name: injected_current} for each electrode.
        # If a contact is not in the given mapping, assign 0.0 by default.
        self.contact_currents = {
            k: float(contact_currents.get(k, 0.0)) for k in (p.name for p in geometry.contacts)
        }

        # (define these BEFORE building conditions)
        # dictionaries will later store derived quantities:
            # contact_flux_densities -> computed per-contact current densities
            # surface_conditions -> objects describing each boundary condition
        self.contact_flux_densities: Dict[str, float] = {}
        self.surface_conditions: Dict[str, SurfaceConditionSummary] = {}

        # -------------------------------
        # 3. Build geometry domains
        # -------------------------------
        self.domains = geometry.build_domains()
        self.spatial_domain = self.domains[geometry.interior_name] # main interior region

        # -------------------------------
        # 4. Build PDE + boundary conditions
        # -------------------------------
        # Assemble all physics constraints for this problem:
        #   • Interior Laplace residual  
        #   • Neumann BCs on electrode contacts  
        #   • Dirichlet BCs on reference / outer surfaces 
        #
        self._conditions = self._build_conditions(outer_bc or {})

        # -------------------------------
        # 5. Initialize PINA base problem
        # -------------------------------
        super().__init__()
        
    @property
    def conditions(self):
        """Return the boundary and PDE conditions dictionary."""
        return self._conditions

    @property
    def geometry(self) -> TowerAGeometry:
        """Return the geometry used to build the problem."""
        return self._geometry

    @property
    def axes(self) -> Axes:
        """Return the spatial axes ordering."""
        return self._axes
    
    def _build_conditions(
        self,
        outer_bc: Mapping[str, Mapping[str, BoundaryValueLike]],
    ) -> Dict[str, Condition]:
        conditions: Dict[str, Condition] = {}

        # PDE residual R_PDE 
        # Enforce the Laplace equation throughout the interior tissue volume.
        # The geometry.interior_name property identifies the main domain (Ω).
        # build_laplace_equation() constructs the physics residual used by PINA.
        conditions["pde"] = Condition(
            domain=self.geometry.interior_name, # Domain where the PDE applies (interior region)
            equation=build_laplace_equation(self.conductivity, self.axes),
        )

        # Contact Neumann conditions:
        # For each electrode patch, create a Neumann BC enforcing the
        # normal current density:  −n·σ∇φ = j_target
            # The j_target value is computed from the electrode's total injected current
            # divided by its surface area → current density (A/m²).
        # This means the normal component of current flow through the surface 
        # equals the prescribed current density j_target.
        for patch in self.geometry.contacts:
            # Iterate over every contact surface defined in the geometry.
            # Each 'patch' object represents one electrode surface with properties

            # Compute the boundary flux density (A/m²) that this contact should impose.
            # 'current_to_flux_density' divides the total electrode current (A)
            # by its surface area (m²) to obtain current density.
            # If no current is specified for this contact, default to 0 A.
            flux_density = bc_utils.current_to_flux_density(
                self.contact_currents.get(patch.name, 0.0), patch.area
            )

            # Construct the Neumann boundary equation:
            equation = build_neumann_equation(
                patch, flux_density, self.conductivity, self.axes
            )

            # used to tag the condition (e.g. "contact:electrode_3").
            domain_name = patch.domain_name("contact")
            key = f"contact_flux:{patch.name}"

            # Create a PINA `Condition` object that ties the Neumann equation
            # to the corresponding surface domain, and register it in the 
            # master dictionary of boundary conditions (`conditions`).
            conditions[key] = Condition(domain=domain_name, equation=equation)
            self.surface_conditions[key] = SurfaceConditionSummary(
                surface=patch,
                condition_type="neumann",
                target=flux_density,
                domain_name=domain_name,
                description=f"I={self.contact_currents.get(patch.name, 0.0)} A",
            )

        # Shank insulation (zero flux)
        for patch in self.geometry.shanks:
            equation = build_neumann_equation(
                patch, 0.0, self.conductivity, self.axes
            )
            domain_name = patch.domain_name("shank")
            key = f"shank_flux:{patch.name}"
            conditions[key] = Condition(domain=domain_name, equation=equation)
            self.surface_conditions[key] = SurfaceConditionSummary(
                surface=patch,
                condition_type="neumann",
                target=0.0,
                domain_name=domain_name,
                description="shank insulation",
            )

        # Outer boundary (Dirichlet or optional Neumann)
        for patch in self.geometry.outers:
            spec = outer_bc.get(patch.name, {"type": "dirichlet", "value": 0.0})
            bc_type = bc_utils.canonicalise_bc_type(spec.get("type", "dirichlet"))
            domain_name = patch.domain_name("outer")
            key = f"outer_bc:{patch.name}"

            if bc_type == "dirichlet":
                value = spec.get("value", 0.0)
                if isinstance(value, (int, float)):
                    equation = FixedValue(float(value), components=["phi"])
                else:
                    equation = build_dirichlet_equation(value)
                description = f"Dirichlet φ={value}"
            else:
                value = spec.get("value", 0.0)
                equation = build_neumann_equation(
                    patch, value, self.conductivity, self.axes
                )
                description = f"Neumann flux={value}"

            conditions[key] = Condition(domain=domain_name, equation=equation)
            self.surface_conditions[key] = SurfaceConditionSummary(
                surface=patch,
                condition_type=bc_type,
                target=value,
                domain_name=domain_name,
                description=description,
            )

        return conditions
    

@dataclass
class BasisTrainingResult:
    """Container for artefacts generated while training a basis field."""

    contact_name: str
    current: float
    problem: TowerALaplaceProblem
    model: nn.Module
    solver: SolverType
    trainer: Trainer



## Tower-A Basis Trainer

workflow for generating unit-current basis fields across the implanted electrode array.

The trainer bundles together geometry, conductivity model, solver class, and coordinate

Strcuture follows PINA framework ( Problem -> Model -> Trainer).

#### Problem Definition:

The build_problem() method constructs a instance for a given current pattern.

#### Model Definition:

The build_model() creates the neural network used to approximate the potential field. By default, this is a smooth multilayer perceptron (MLP 6 layers x 128 units) with hyperbolic tangent activations, 

chosen for its ability to capture continuous gradients in space. 

Fourier feature mappings can optionally be included to improve representation of spatial detail or geometric irregularity.

#### Discretisation

The discretise() routine samples collocation points in both the interior domain and along boundaries. Interior samples enforce the PDE residual, while boundary samples enforce Neumann or Dirichlet constraints on contact, shank, and outer surfaces. Defaults use Latin hypercube sampling for the interior and random sampling on boundaries to balance coverage and variability. This step essentially defines the "training data" for the physics loss.

####  Training Procedure

The train_basis() method orchestrates the full training process for one electrode at a time:

Training continues until convergence producing a trained model that approximates ϕ(x) for that electrode. The output is packaged into a BasisTrainingResult object containing the problem definition, model, solver, and training logs.

Reuse and Composition

Once all contacts have been trained, their basis fields can be linearly superposed in downstream modules (e.g., Tower-A’s field encoder or Tower-B’s response model) to reconstruct the potential field for any stimulation vector. This modular training approach allows individual basis functions to be recalibrated or extended without retraining the entire network ensemble.

In [ ]:
class TowerABasisTrainer:
    """Utility to train unit-current basis fields with sensible defaults."""

    def __init__(
        self,
        geometry: TowerAGeometry,
        conductivity: ConductivityLike,
        solver_cls: SolverClass = PINN,
        axes: Axes = DEFAULT_AXES,
    ) -> None:
        self.geometry = geometry
        self.conductivity = conductivity
        self.solver_cls = solver_cls
        self.axes = axes

    # ------------------------------------------------------------------
    # Problem/model construction helpers
    # ------------------------------------------------------------------
    def build_problem(
        self,
        contact_currents: Mapping[str, float],
        outer_bc: Optional[Mapping[str, Mapping[str, BoundaryValueLike]]] = None,
    ) -> TowerALaplaceProblem:
        """Instantiate a :class:`TowerALaplaceProblem` for the given pattern."""
        return TowerALaplaceProblem(
            geometry=self.geometry,
            conductivity=self.conductivity,
            contact_currents=contact_currents,
            outer_bc=outer_bc,
            axes=self.axes,
        )

    def build_model(
        self,
        n_layers: int = 6,
        hidden_size: int = 128,
        activation: type[nn.Module] = nn.Tanh,
        fourier_features: Optional[nn.Module] = None,
    ) -> nn.Module:
        """Return the default smooth MLP used for basis fields."""
        model = FeedForward(
            input_dimensions=len(self.axes),
            output_dimensions=1,
            inner_size=hidden_size,
            n_layers=n_layers,
            func=activation,
        )
        if fourier_features is not None:
            # Simple wrapper: pass inputs through features before MLP
            return nn.Sequential(fourier_features, model)
        return model

    # ------------------------------------------------------------------
    # Discretisation
    # ------------------------------------------------------------------
    def discretise(
        self,
        problem: TowerALaplaceProblem,
        interior_points: int = 50_000,
        contact_points: Union[int, Mapping[str, int]] = 8_192,
        shank_points: Union[int, Mapping[str, int]] = 4_096,
        outer_points: Union[int, Mapping[str, int]] = 4_096,
        interior_mode: str = "latin",
        boundary_mode: str = "random",
    ) -> None:
        """Sample all domains with reasonable defaults."""

        def _sample_domain(n: int, mode: str, domain: str) -> None:
            if n <= 0:
                n = 1  # ensure at least one point per domain
            problem.discretise_domain(n=n, mode=mode, domains=domain)

        _sample_domain(interior_points, interior_mode, problem.geometry.interior_name)

        def _resolve(mapping_or_int: Union[int, Mapping[str, int]], name: str, default: int) -> int:
            if isinstance(mapping_or_int, Mapping):
                return int(mapping_or_int.get(name, default))
            return int(mapping_or_int)

        for patch in problem.geometry.contacts:
            n_pts = _resolve(contact_points, patch.name, int(contact_points))
            _sample_domain(n_pts, boundary_mode, patch.domain_name("contact"))
        for patch in problem.geometry.shanks:
            n_pts = _resolve(shank_points, patch.name, int(shank_points))
            _sample_domain(n_pts, boundary_mode, patch.domain_name("shank"))
        for patch in problem.geometry.outers:
            n_pts = _resolve(outer_points, patch.name, int(outer_points))
            _sample_domain(n_pts, boundary_mode, patch.domain_name("outer"))

    # ------------------------------------------------------------------
    # Training loop
    # ------------------------------------------------------------------
    def train_basis(
        self,
        contact_name: str,
        current: float = 1e-6,
        *,
        outer_bc: Optional[Mapping[str, Mapping[str, BoundaryValueLike]]] = None,
        solver_kwargs: Optional[Dict] = None,
        trainer_kwargs: Optional[Dict] = None,
        model: Optional[nn.Module] = None,
        discretisation_kwargs: Optional[Dict] = None,
    ) -> BasisTrainingResult:
        """Train a unit-current basis field for the specified contact."""
        if contact_name not in {patch.name for patch in self.geometry.contacts}:
            raise ValueError(f"Unknown contact name {contact_name!r}.")

        contact_currents = {patch.name: 0.0 for patch in self.geometry.contacts}
        contact_currents[contact_name] = current

        problem = self.build_problem(contact_currents, outer_bc=outer_bc)

        discretisation_kwargs = discretisation_kwargs or {}
        self.discretise(problem, **discretisation_kwargs)

        model = model or self.build_model()

        solver_kwargs = solver_kwargs or {}
        solver = self.solver_cls(problem=problem, model=model, **solver_kwargs)

        trainer_kwargs = {
            "max_epochs": 2_000,
            "enable_model_summary": False,
            "accelerator": "cpu",
        } | (trainer_kwargs or {})
        trainer = Trainer(
            solver=solver,
            train_size=1.0,
            val_size=0.0,
            test_size=0.0,
            batch_size=None,
            **trainer_kwargs,
        )

        trainer.train()

        return BasisTrainingResult(
            contact_name=contact_name,
            current=current,
            problem=problem,
            model=model,
            solver=solver,
            trainer=trainer,
        )
if __name__ == "__main__":
    print("Tower-A PINN field module loaded successfully ")